In [145]:
import csv
import re
import os
import pandas as pd
import numpy as np
import xlsxwriter
import shutil

from datetime import datetime, date

In [146]:
file_root = './files'
church_export_root = './church'

pricing_breakdown = {
    'student': {
        '05-17-23': 175,
        '06-01-23': 225,
    },
    'chaperone': {
        'High School Camp': 125,
        'Middle School Camp': 125,
        'Both Camps': 200,
    }
}

church_list = [
    "Ashland City Faith Community",
    "Bell Road",
    "Bent Knee Cowboy",
    "Body of Christ",
    "Bread of Life Community",
    "Bridge Point",
    "Brookhaven First",
    "Clarksdale",
    "Clarksville First Community of Hope",
    "Clarksville Grace",
    "Clarksville Hope Riders Biker Church",
    "Cleveland First",
    "Columbia First",
    "Columbia Grace",
    "Columbus First",
    "Community Fellowship",
    "Concord Community",
    "Covington",
    "Crossroads",
    "Crossroads Community",
    "Crystal Springs",
    "DaySpring Community",
    "Dickson Lighthouse",
    "Donelson",
    "Dover",
    "Elmore",
    "Erin",
    "Fellowship",
    "Fly",
    "Foundry",
    "Franklin Community Faith",
    "Fulton First",
    "Gallatin First",
    "Gateway Community",
    "Glencliff Hispanic",
    "Gloster",
    "Goodlettsville",
    "Gray's Chapel",
    "Grenada First",
    "Griffin's Chapel",
    "Gulf Coast Family",
    "Harmony",
    "Hattiesburg First",
    "Hazkehurst",
    "Hendersonville",
    "Hermitage",
    "Highland",
    "Hillside Community",
    "Igelsia Del Nazareno Eben-Ezer",
    "Igelsia Hispana Betel de Gallatin",
    "Jason Chapel",
    "Joelton",
    "Jones Chapel",
    "Kingston Springs",
    "Laurel Bresee",
    "Laurel First",
    "Lewisburg First",
    "Libertad en Cristo",
    "Lifepointe Community",
    "Long Creek",
    "Madison",
    "Madison Hispanic",
    "Magnolia",
    "Memphis Calvary",
    "Memphis Eastside",
    "Memphis Emmanuel",
    "Memphis Friendship",
    "Memphis Grace",
    "Memphis Holiness Tabernacle",
    "Memphis Hope Tabernacle",
    "Memphis Refreshing Springs",
    "Meridian Fitkins Memorial",
    "Meridian Northside",
    "Millington",
    "Mount Wesley-Akin Chapel",
    "Nashville Blakemore",
    "Nashville Christ",
    "Nashville First (Main Campus)",
    "Nashville First (Bethel Campus)",
    "Nashville First (Calvary) Korean",
    "Nashville Grace",
    "Nashville Inglewood",
    "Nashville Kaleo",
    "Nashville Trinity",
    "New Albany First",
    "New Vision",
    "Old Hickory",
    "Oxford",
    "Palabra de Vida",
    "Paris",
    "Park Lane",
    "Pearson Chapel",
    "Pine Hill",
    "Pleasant Point",
    "Portland",
    "Prentiss",
    "Quitman",
    "Ripley Hispanic",
    "Rosebloom",
    "Savannah",
    "Sentobia N.O. Nabors Memorial",
    "Spring Hill Restoration Community",
    "Springfield First",
    "Starkville Wesley Community",
    "Trevecca Community",
    "Union",
    "Vicksburg First/Vicksburg Hispanic",
    "Victory Praise and Worship Center",
    "Waverly",
    "Way of the Cross",
    "Waynesboro Trinity",
    "Westwind Community of Faith",
    "Woodlawn Community"
]

In [147]:
files = os.listdir(file_root)
files = [f for f in files if os.path.isfile(f'{file_root}/{f}') and f != '.DS_Store'] 
files

['Camp_Registration_-_20232023-05-16_18_28_22.csv']

In [148]:
def create_directory(process):
    outdir = f'./{process}/{date.today()}/'
    
    if not os.path.exists(outdir):
        os.mkdir(outdir)

In [149]:
def write_csv(df, df_name, process, filename):
    create_directory(process)
    if df_name not in write_file_exlusion:
        df.to_csv(f'{filename}.csv')

In [150]:
def write_excel(df, df_name, process, filename, indexed=False):
    create_directory(process)
    if df_name not in write_file_exlusion:
        df.to_excel(f'{filename}.xlsx', index=indexed, engine='xlsxwriter')

In [151]:
def move_to_processed(filename):
    src_file = f'{file_root}/{filename}'
    dst_file = f'./processed/{filename}'
    
    shutil.move(src_file, dst_file)

In [152]:
def strip_filename_from_download_date(filename):
    filename_parts = filename.split('_-_')
    return filename_parts[0]

strip_filename_from_download_date(files[0])

'Camp_Registration'

In [153]:
def get_price(row):
    if row['Are you a student or a chaperone'] == 'Chaperone':
        return pricing_breakdown['chaperone'][row['At which camp will you be a chaperone?']]
    
    submission_date_string = row['Submission Date']
    price_dates = pricing_breakdown['student'].keys()
    
    for price_date_string in price_dates:
        submission_date = datetime.strptime(submission_date_string, '%b %d, %Y')
        price_date = datetime.strptime(price_date_string, '%m-%d-%y')
        if submission_date <= price_date:
            return pricing_breakdown['student'][price_date_string]

In [154]:
# def camp
def create_church_info_sheets(file, filename):
    df_original = pd.read_csv(f'{file_root}/{file}')

    df = df_original.copy(deep=True)
    # df['Camp'] = df.apply(lambda x: x['At which camp will you be a chaperone?'] if x['At which camp will you be a chaperone?'] else x['Which Camp?'])
    df['Camp'] = df['At which camp will you be a chaperone?'].combine_first(df['Which Camp?'])

    cols = df_original.columns;
    cols = [*cols, 'Camp']
    df = df.loc[:, cols].fillna('')
    df = df.groupby(['What church are you a part of?', 'First Name', 'Last Name'], group_keys=False).apply(lambda x: x)
    
    df.sort_values('Last Name')
    df.reset_index(inplace=True)
    # df['What church are you a part of?'] = df['What church are you a part of?'].apply(lambda x: x.strip())
    for church in church_list:
        df_church = df.loc[df['What church are you a part of?'] == church]
        if not df_church.empty:
            df_church['Price'] = df_church.apply(lambda x: get_price(x), axis=1)
            df_church['Paid'] = df_church['Payment'].apply(lambda x: int(re.findall("[0-9]+", x)[0]) if re.findall("[0-9]+", x) else 0)
            df_church['Total Due'] = df_church.apply(lambda x: x['Price'] - x['Paid'], axis=1)
            df_church.rename(columns = {"First Name.1": "Guardian First Name", "Last Name.1": "Guardian Last Name","First Name.2": "Youth Leader First Name", "Last Name.2": "Youth Leader Last Name"}, inplace = True)

            df_church.reset_index(inplace=True)

            df_church.loc['Total',:] = df_church.sum(axis=0, numeric_only=True)
            
            export_path = f"{church_export_root}/{date.today()}/{church.replace(' ', '_').lower()}"
            write_excel(df_church, church, 'church', export_path)


create_church_info_sheets(files[0], strip_filename_from_download_date(files[0]))

/var/folders/7c/1cwmtz9d76g0tk04g82t4xzw0000gn/T/ipykernel_7236/4257252877.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_church['Price'] = df_church.apply(lambda x: get_price(x), axis=1)
/var/folders/7c/1cwmtz9d76g0tk04g82t4xzw0000gn/T/ipykernel_7236/4257252877.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_church['Paid'] = df_church['Payment'].apply(lambda x: int(re.findall("[0-9]+", x)[0]) if re.findall("[0-9]+", x) else 0)
/var/folders/7c/1cwmtz9d76g0tk04g82t4xzw0000gn/T/ipykernel_7236

In [155]:
for file in files:
    filename = strip_filename_from_download_date(file)

    create_church_info_sheets(file, filename)
    move_to_processed(file)

/var/folders/7c/1cwmtz9d76g0tk04g82t4xzw0000gn/T/ipykernel_7236/4257252877.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_church['Price'] = df_church.apply(lambda x: get_price(x), axis=1)
/var/folders/7c/1cwmtz9d76g0tk04g82t4xzw0000gn/T/ipykernel_7236/4257252877.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_church['Paid'] = df_church['Payment'].apply(lambda x: int(re.findall("[0-9]+", x)[0]) if re.findall("[0-9]+", x) else 0)
/var/folders/7c/1cwmtz9d76g0tk04g82t4xzw0000gn/T/ipykernel_7236